In [6]:
import PySimpleGUI as sg
import serial
import sys
import numpy as np
import os
from simple_pid import PID
import minimalmodbus
import pandas as pd
import threading
import datetime


---

In [3]:
class ValveCtrlSystem():
    def __init__(self):
        
        # инициализация ПИД регуляторов для клапанов
        
        self.target = 60
        
        self.pid_pos_1 = PID(1500, 50, 1, setpoint=self.target)
        self.pid_pos_1.output_limits = (0, 11000) 
        self.pid_pos_2 = PID(1500, 50, 1, setpoint=self.target)
        self.pid_pos_2.output_limits = (0, 11000) 
        
    def set_target(self, target):
        
        # установка уставки
        
        self.target = target
        self.pid_pos_1.setpoint = target
        self.pid_pos_2.setpoint = target
        
    def ctrl_signal(self, current_value):
        
        # расчет и выдача управляющего сигнала
        
        pos_1_ctrl_signal = int(self.pid_pos_1(current_value))
        pos_2_ctrl_signal = int(11000 - self.pid_pos_2(current_value))
        return pos_1_ctrl_signal, pos_2_ctrl_signal

In [4]:
class HumidityGenerator():
    def __init__(self, COM_PORT):
        
        try:
            
            self.instrument = minimalmodbus.Instrument(COM_PORT, 0xD7)
        
            self.instrument.serial.baudrate = 19200
            self.instrument.serial.bytesize = 8
            self.instrument.serial.stopbits = 1
            
        except Exception as e: 
            print('Проверьте работоспособность COM-порта и выполните инициализацию класса снова. ' + 'Текст ошибки: ' + str(e))
        
        self.loggs = []
        
        # влажность
        self.humidity = 0.0
        
        # расход
        self.g_1 = 0.0
        self.g_2 = 0.0
        self.g_tot = 0.0
        
        # температура
        self.temp = 0.0
        
        # время    
        self.t = 0.0
        
        # для отправки
        
        self.pos_1 = 0
        self.pos_2 = 0
        self.t_discr = 0
        self.t_work = 0
        
    def update(self, recived_data, target):
        
        # обновление значений 
        
        self.humidity = recived_data[0]

        self.g_1 = recived_data[2]

        self.g_2 = recived_data[3]

        self.g_tot = recived_data[2] + recived_data[3]

        self.temp = recived_data[1]

        self.t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # запись строки в логи
        
        self.loggs.append([target,
                           self.humidity, 
                           self.g_1,
                           self.g_2, 
                           self.g_tot, 
                           self.temp, 
                           self.t,
                           self.pos_1, 
                           self.pos_2, 
                           self.t_discr,
                           self.t_work])
        
    def get_params(self):
        
        # получение значений с генератора
        
        try:
            recived_data = self.instrument.read_registers(0, 4, functioncode=3)
            return recived_data
        except:
            return [0, 0, 0, 0]
        
    def send_params(self, send_data):
        
        # отправка значений в генератор
        
        self.pos_1 = send_data[0]
        self.pos_2 = send_data[1]
        self.t_discr = send_data[2]
        self.t_work = send_data[3]
        
        self.instrument.write_registers(0, send_data)
        
    def write_loggs(self):
        
        dir = os.path.abspath(os.curdir)
        
        name = 'loggs_'+ str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        
        # запись логов в эксель
        
        df = pd.DataFrame(data=self.loggs, columns = ['Влажность-уставка, %',
                                                      'Влажность, %',
                                                      'Расход 1, мл',
                                                      'Расход 2, мл',
                                                      'Сумма расх., мл',
                                                      'Температура, гр.Цел.',
                                                      'Время',
                                                      'Клапан 1, полож', 
                                                      'Клапан 2, полож', 
                                                      'Время дискр, мс', 
                                                      'Время вкл, мс'])
        df.to_excel(dir + '/' + name + '.xlsx')
        return name + '.xlsx'

In [7]:
def daemon(humidity_generator, valve_ctrl_system, delay, event_thr):
    while True:
        # получение данных
        recived_data = humidity_generator.get_params()
        current_humidity = recived_data[0]
    
        # обновление данных в классе HumidityGenerator
    
        humidity_generator.update(recived_data, valve_ctrl_system.target)
    
        # расчет управляющего сигнала
    
        pos_1_ctrl_signal, pos_2_ctrl_signal = valve_ctrl_system.ctrl_signal(current_humidity)
    
        # отправка данных
    
        send_data = [pos_1_ctrl_signal, pos_2_ctrl_signal, 10000, 0]
    
        humidity_generator.send_params(send_data)
        
        print(humidity_generator.t,
              ' Влажность, %: ', current_humidity,
              ' Уставка, %: ', valve_ctrl_system.target,
              ' Суммарный расход, мл:', humidity_generator.g_tot)

    
        event_thr.wait(delay)
        if event_thr.is_set():
            break

In [ ]:
valve_ctrl_system = ValveCtrlSystem()
humidity_generator = HumidityGenerator(COM_PORT)
event_thr = threading.Event()
event_thr.clear()
thr1 = threading.Thread(target = daemon, args=(humidity_generator, valve_ctrl_system, 1, event_thr)).start()

---